#  **Pandas Assingment-4**

(Based on IPL matches and deliveries dataset)


Matches- <a>https://www.kaggle.com/nowke9/ipldata#matches.csv</a>


Deliveries- <a>https://www.kaggle.com/nowke9/ipldata#deliveries.csv<a>

In [0]:
import pandas as pd
matches=pd.read_csv('matches (1).csv')
delivery=pd.read_csv('deliveries(1).csv')

In [0]:
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [0]:
delivery.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,NaN,NaN,NaN


In [0]:
#  Q1.- Make a pivot table of top 10 bowlers(in terms of wickets) with over(1-20) showing no. of wickets.  [Dificult-]

In [0]:
# Extracting the top 10 bowlers in terms of wickets taken


# Extracting the balls in which wickets were taken and the wicket was given to the bowler
mask= (delivery['dismissal_kind']=='caught') | (delivery['dismissal_kind']=='bowled') | (delivery['dismissal_kind']=='lbw') | (delivery['dismissal_kind']=='caught and bowled') | (delivery['dismissal_kind']=='stumped')
wickets=delivery[mask]

# Counting the number of wickets, sorting in descending order and extracting the names of the top 10 bowlers
best_bowlers=wickets.groupby('bowler').count()['ball'].sort_values(ascending=False).head(10).index.tolist()       
print('Top 10 Bowlers: ',best_bowlers)


# Filtering the data for only these bowlers
top_bowlers_data=wickets[wickets['bowler'].isin(best_bowlers)]

# Collecting over-wise data of each bowler by counting the number of balls in the whole IPL
over_wise_data=top_bowlers_data.groupby(['bowler','over']).count()['ball'].reset_index()

# Making the pivot table
over_wise_data.pivot_table(index='bowler',columns='over',values='ball')

Top 10 Bowlers:  ['SL Malinga', 'A Mishra', 'PP Chawla', 'Harbhajan Singh', 'DJ Bravo', 'B Kumar', 'R Ashwin', 'SP Narine', 'UT Yadav', 'RA Jadeja']


over,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
bowler,,,,,,,,,,,,,,,,,,,,
A Mishra,1.0,NaN,NaN,4.0,1.0,NaN,15.0,9.0,20.0,10.0,13.0,13.0,10.0,16.0,6.0,14.0,7.0,6.0,5.0,6.0
B Kumar,17.0,7.0,12.0,3.0,4.0,3.0,NaN,3.0,NaN,NaN,1.0,1.0,1.0,1.0,2.0,12.0,8.0,16.0,19.0,23.0
DJ Bravo,NaN,1.0,2.0,2.0,1.0,4.0,7.0,4.0,6.0,2.0,3.0,6.0,7.0,7.0,5.0,13.0,9.0,29.0,10.0,29.0
Harbhajan Singh,5.0,5.0,1.0,7.0,4.0,8.0,12.0,13.0,8.0,12.0,12.0,13.0,6.0,12.0,5.0,7.0,8.0,6.0,4.0,1.0
PP Chawla,NaN,2.0,1.0,1.0,2.0,10.0,6.0,8.0,10.0,9.0,15.0,8.0,15.0,11.0,14.0,11.0,11.0,6.0,2.0,7.0
R Ashwin,7.0,1.0,5.0,5.0,9.0,8.0,2.0,9.0,5.0,5.0,10.0,10.0,7.0,7.0,7.0,15.0,4.0,7.0,1.0,1.0
RA Jadeja,NaN,NaN,NaN,NaN,2.0,4.0,10.0,4.0,9.0,9.0,7.0,8.0,7.0,9.0,13.0,3.0,11.0,5.0,4.0,3.0
SL Malinga,11.0,11.0,2.0,3.0,5.0,5.0,1.0,NaN,1.0,2.0,1.0,4.0,9.0,3.0,4.0,18.0,14.0,25.0,32.0,19.0
SP Narine,NaN,2.0,2.0,3.0,9.0,8.0,NaN,3.0,4.0,2.0,10.0,5.0,4.0,6.0,7.0,9.0,12.0,13.0,12.0,11.0


In [0]:
# Q2.- Make a function for batting profile of any batsman:-                                              [Medium+]


Ex- 

V Kohli:

Innings:  

Total Runs: 

No. of MOM(Man of the match):

No. of boundries:

In [0]:
def batting_profile(batsman):

  if batsman not in delivery['batsman'].values:
    return 'Batsman Not Found'
  # Getting the number of innings played by the batsman
  innings=delivery[(delivery['batsman']==batsman) | (delivery['non_striker']==batsman)].drop_duplicates(subset=['match_id'])
  no_innings=innings.shape[0]

  # Getting total runs of a batsman
  batsman_striker=delivery[delivery['batsman']==batsman]
  total_runs=batsman_striker['batsman_runs'].sum()

  # Getting no. of MOM
  mom=matches[matches['player_of_match']==batsman].shape[0] 

  # No of boundaries
  boundaries=batsman_striker[(batsman_striker['batsman_runs']==4) | (batsman_striker['batsman_runs']==6)].shape[0]

  # Building Profile String
  profile= """Batsman: {},   Innings Played: {},  Total Runs: {},  No. of Man of the Matches: {},   No. of Boundaries: {}""".format(batsman,no_innings,total_runs,mom,boundaries)
  return profile


In [0]:
batting_profile('V Kohli')

'Batsman: V Kohli,   Innings Played: 169,  Total Runs: 5434,  No. of Man of the Matches: 12,   No. of Boundaries: 673'

In [0]:
batting_profile('M Dhoni')

'Batsman Not Found'

In [0]:
batting_profile('MS Dhoni')

'Batsman: MS Dhoni,   Innings Played: 170,  Total Runs: 4477,  No. of Man of the Matches: 17,   No. of Boundaries: 504'

In [0]:
batting_profile('AB de Villiers')

'Batsman: AB de Villiers,   Innings Played: 142,  Total Runs: 4428,  No. of Man of the Matches: 20,   No. of Boundaries: 571'

In [0]:
# Q3.- Write a function to return top-4 batsman at a venue. Take venue as input.

In [0]:
def venue_top_batsmen(venue):
  if venue not in matches['city'].values:
    return "Venue Not Found"
  # Extracting the match ids of matches played in Venue from matches dataset
  venue_match_ids=matches[matches['city']==venue]['id']

  # Filtering the delivery dataset based on the match ids received from above
  venue_matches=delivery[delivery['match_id'].isin(venue_match_ids)]

  # Finding the top 4 batsmen by adding up the batsman runs
  return venue_matches.groupby('batsman').sum()['batsman_runs'].sort_values(ascending=False).head(4).index.tolist()


In [0]:
venue_top_batsmen('Kolkata')

['G Gambhir', 'RV Uthappa', 'YK Pathan', 'AD Russell']

In [0]:
venue_top_batsmen('kokata')

'Venue Not Found'

In [0]:
venue_top_batsmen('Chennai')

['SK Raina', 'MS Dhoni', 'M Vijay', 'MEK Hussey']

In [0]:
venue_top_batsmen('Bangalore')

['V Kohli', 'CH Gayle', 'AB de Villiers', 'JH Kallis']

In [0]:
# Q4.- No. of matches per season played in Mumbai.

In [0]:
matches_in_mumbai=matches[matches['city']=='Mumbai'].groupby('season').count()['id']
print("Matches Played in MUMBAI in every season: ")
print(matches_in_mumbai)
print('Average number of matches played in MUMBAI per season: ',matches_in_mumbai.mean())

Matches Played in MUMBAI in every season: 
season
2008    10
2010    13
2011    16
2012     8
2013     8
2014     7
2015    11
2016     4
2017     8
2018     9
2019     7
Name: id, dtype: int64
Average number of matches played in MUMBAI per season:  9.181818181818182


In [0]:
# Q5. - Top-3 Player in terms of most no. of catches.

In [0]:
#| (delivery['dismissal_kind']=='caught and bowled')
fielder_catches=delivery[(delivery['dismissal_kind']=='caught') ]['fielder'].value_counts()
bowler_catches=delivery[delivery['dismissal_kind']=='caught and bowled']['bowler'].value_counts()

In [0]:
fielder_catches.add(bowler_catches,fill_value=0).sort_values(ascending=False).head(3)

KD Karthik    109.0
SK Raina      102.0
MS Dhoni       98.0
dtype: float64